In [1]:
import numpy as np
from bloqade import squin
from bloqade.pyqrack import StackMemorySimulator
from kirin.dialects.ilist import IList

In [ ]:
@squin.kernel
def steane_steane_qec_rounds3_measure_data():
    q = squin.qalloc(15)
    verify = q[14]

    data = IList([q[0], q[1], q[2], q[3], q[4], q[5], q[6]])
    aux  = IList([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

    results = []

    def encode_figA2_logical_zero(block7, verify_q, results, do_verify=True):
        squin.broadcast.reset(block7)
        squin.reset(verify_q)

        squin.h(block7[0])  # 1
        squin.h(block7[1])  # 2
        squin.h(block7[3])  # 4

        # first 8 CNOTs
        squin.cx(block7[0], block7[4]) 
        squin.cx(block7[1], block7[2])
        squin.cx(block7[3], block7[5])
        squin.cx(block7[0], block7[6])
        squin.cx(block7[3], block7[4])
        squin.cx(block7[1], block7[5])
        squin.cx(block7[0], block7[2])
        squin.cx(block7[5], block7[6])

        # Verification
        if do_verify:
            squin.reset(verify_q)
            squin.cx(block7[4], verify_q)  
            squin.cx(block7[2], verify_q) 
            squin.cx(block7[5], verify_q)
            results.append(squin.measure(verify_q))
            squin.reset(verify_q)

    # Encode DATA |0>_L (with verification)
    encode_figA2_logical_zero(data, verify, results, do_verify=True)

    for _ in range(3):

        # ---- Z syndrome half-cycle (detect X errors) ----
        encode_figA2_logical_zero(aux, verify, results, do_verify=True)
        squin.broadcast.h(aux)  # |0>_L -> |+>_L

        for i in range(7):
            squin.cx(data[i], aux[i])     # data -> aux (transversal)

        measZ = squin.broadcast.measure(aux)  # Z-basis
        results.extend(measZ)
        squin.broadcast.reset(aux)

        # ---- X syndrome half-cycle (detect Z errors) ----
        encode_figA2_logical_zero(aux, verify, results, do_verify=True)

        for i in range(7):
            squin.cx(aux[i], data[i])     # aux -> data (inverted transversal)

        squin.broadcast.h(aux)            # X-basis measurement
        measX = squin.broadcast.measure(aux)
        results.extend(measX)
        squin.broadcast.reset(aux)

    # Final data readout
    final_data = squin.broadcast.measure(data)
    results.extend(final_data)

    return results


In [ ]:
import bloqade.tsim
from collections import Counter
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit

@squin.kernel
def main():
    steane_steane_qec_rounds3_measure_data()

cirq_main = emit_circuit(main)

# add noise here later
#---------------

#---------------

# Cirq to Squin
squin_lowered = load_circuit(cirq_main)
tsim_circ = bloqade.tsim.Circuit(squin_lowered)
tsim_circ.diagram("timeline-svg", height=300)

sampler = tsim_circ.compile_sampler()
samples = sampler.sample(shots=200)

print("samples shape:", samples.shape)


samples shape: (200, 56)


In [ ]:
import numpy as np

# Steane Z-checks correspond to Hamming parity checks:
# (0,1,2,4), (0,1,3,5), (0,2,3,6)
# Z-check syndrome (3 bits) -> which data qubit had an X error (single-error decoder)

STEANE_SYNDROME_TO_Q = {
    (0,0,0): None,  # + + +  -> no error
    (1,0,0): 0,     # - + +  -> X1
    (0,0,1): 1,     # + + -  -> X2
    (1,0,1): 2,     # - + -  -> X3
    (0,1,0): 3,     # + - +  -> X4
    (1,1,0): 4,     # - - +  -> X5
    (0,1,1): 5,     # + - -  -> X6
    (1,1,1): 6,     # - - -  -> X7
}


def parse_sample(row, rounds=3):
    row = [int(b) for b in row]
    expected = 1 + rounds * 16 + 7
    if len(row) != expected:
        raise ValueError(f"Expected {expected} bits per shot, got {len(row)}")

    vData = row[0]
    offset = 1

    vZ_list, aux_Z = [], []
    vX_list, aux_X = [], []

    for r in range(rounds):
        vZ = row[offset]; offset += 1
        z_bits = row[offset:offset+7]; offset += 7

        vX = row[offset]; offset += 1
        x_bits = row[offset:offset+7]; offset += 7

        vZ_list.append(vZ); aux_Z.append(z_bits)
        vX_list.append(vX); aux_X.append(x_bits)

    data = row[offset:offset+7]
    return data, aux_Z, aux_X, vData, vZ_list, vX_list


STEANE_CHECKS = [
    (0, 2, 4, 6),  # S^(1): 1,3,5,7
    (3, 4, 5, 6),  # S^(2): 4,5,6,7
    (1, 2, 5, 6),  # S^(3): 2,3,6,7
]


def build_syndrome_to_q(checks, n=7):
    # syndrome bit convention: 1 means that check is flipped (odd parity)
    m = {(0,0,0): None}
    for i in range(n):
        s = tuple(1 if i in chk else 0 for chk in checks)
        m[s] = i
    return m

STEANE_SYNDROME_TO_Q = build_syndrome_to_q(STEANE_CHECKS)


def parity(bits, idxs):
    p = 0
    for i in idxs:
        p ^= int(bits[i])
    return p

def syndrome_from_aux_measurements(aux7_bits):
    # 3-bit syndrome
    return [parity(aux7_bits, chk) for chk in STEANE_CHECKS]

def postselect_on_flags(flags):
    # reject if any flag fired in any round
    for r in range(len(flags)):
        if any(flags[r]):
            return False
    return True

def correct_single_X_from_last_Z_syndrome(data_bits, last_Z_syndrome3):
    if len(last_Z_syndrome3) != 3:
        return data_bits

    s = tuple(int(x) for x in last_Z_syndrome3)
    qidx = STEANE_SYNDROME_TO_Q.get(s, None)
    if qidx is None:
        return data_bits
    data_bits = data_bits.copy()
    data_bits[qidx] ^= 1
    return data_bits

def correct_single_Z_from_last_X_syndrome(z_frame, last_X_syndrome3):
    """
    X-syndrome detects Z errors, tracked in a Pauli frame: z_frame[i] = 1 means a Z correction is pending on qubit i.
    """
    if len(last_X_syndrome3) != 3:
        return z_frame

    s = tuple(int(x) for x in last_X_syndrome3)
    qidx = STEANE_SYNDROME_TO_Q.get(s, None)
    if qidx is None:
        return z_frame

    z_frame = z_frame.copy()
    z_frame[qidx] ^= 1
    return z_frame


def logical_Z_parity(data_bits):
    p = 0
    for b in data_bits:
        p ^= int(b)
    return p  # 1 means logical Z = -1

def _is_trivial_syndrome(s):
    return tuple(s) == (0,0,0)

def verified_ok(vData, vZ_list, vX_list):
    if vData == 1:
        return False
    if any(v == 1 for v in vZ_list):
        return False
    if any(v == 1 for v in vX_list):
        return False
    return True

def logical_error_rate(samples, rounds=3, postselect=False):
    kept = 0
    successes = 0

    for row in samples:
        data, auxZ, auxX, vData, vZ_list, vX_list = parse_sample(row, rounds=rounds)

        if postselect and not verified_ok(vData, vZ_list, vX_list):
            continue

        data_corr = data
        z_frame = [0]*7  # track Z corrections


        # Apply Steane Pauli-frame updates round-by-round
        for r in range(rounds):
            z_synd = syndrome_from_aux_measurements(auxZ[r])  # detects X errors

            # X stabilizers -> detect Z errors
            x_synd = syndrome_from_aux_measurements(auxX[r])  

            if postselect:
                if not _is_trivial_syndrome(z_synd) or not _is_trivial_syndrome(x_synd):
                    # discard this shot
                    data_corr = None
                    break

            data_corr = correct_single_X_from_last_Z_syndrome(data_corr, z_synd)
            z_frame = correct_single_Z_from_last_X_syndrome(z_frame, x_synd)

        if data_corr is None:
            continue

        # If logical_Z_parity returns 1 for "correct logical eigenvalue", this is success.
        successes += (logical_Z_parity(data_corr) == 0)

        kept += 1

    if kept == 0:
        return np.nan, 0

    p_success = successes / kept
    p_error = 1 - p_success
    return p_error, kept


In [ ]:
import bloqade.tsim
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit
from collections import Counter

# Squin kernel to Cirq
cirq_main = emit_circuit(main)

def sample_with_noise(noise_model, shots=2000):
    # 2) Add noise (Cirq circuit annotated/transformed)
    cirq_noisy = noise.transform_circuit(cirq_main, model=noise_model)

    # 3) Convert back to SQUIN
    squin_noisy = load_circuit(cirq_noisy)

    # 4) Convert to tsim and sample
    tsim_circ = bloqade.tsim.Circuit(squin_noisy)
    sampler = tsim_circ.compile_sampler()
    samples = sampler.sample(shots=shots)

    # 5) Histogram full bitstrings
    hist = Counter("".join(str(int(b)) for b in row) for row in samples)

    # 6) Logical error rates
    p_no_ps, kept_no_ps = logical_error_rate(samples, rounds=3, postselect=False)
    p_ps, kept_ps = logical_error_rate(samples, rounds=3, postselect=True)

    return hist, tsim_circ, (p_no_ps, kept_no_ps), (p_ps, kept_ps)




In [ ]:
import matplotlib.pyplot as plt

scales = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
shots = 5000

p_no_ps_list = []
p_ps_list = []
kept_frac_list = []

for s in scales:
    model = noise.GeminiOneZoneNoiseModel(scaling_factor=s)
    hist, _, (p_no_ps, kept_no_ps), (p_ps, kept_ps) = sample_with_noise(model, shots=shots)
    
    print("scale:", s, " kept_ps:", kept_ps, " p_ps:", p_ps)
    p_no_ps_list.append(p_no_ps)
    p_ps_list.append(p_ps)

    # kept_ps is the number of shots that survived Steane syndrome post-selection
    kept_frac_list.append(kept_ps / shots)

# Plot logical error vs scale (no postselect vs postselect)
plt.figure()
plt.plot(scales, p_no_ps_list, marker="o", label="no post-selection")
plt.plot(scales, p_ps_list, marker="o", label="Steane syndrome post-selection")
plt.xlabel("Noise scaling_factor")
plt.ylabel("Logical error rate (logical Z)")
plt.title("Steane QEC memory: logical error vs physical noise scale")
plt.grid(True)
plt.legend()
plt.show()

# Plot acceptance rate under post-selection
plt.figure()
plt.plot(scales, kept_frac_list, marker="o")
plt.xlabel("Noise scaling_factor")
plt.ylabel("Kept fraction (Steane syndrome post-selection)")
plt.title("Post-selection acceptance vs noise")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import bloqade.stim
from bloqade.cirq_utils import load_circuit

def run_error_fraction(cirq_circuit, shots=500):
    squin_circuit = load_circuit(cirq_circuit)
    stim_circuit = bloqade.stim.Circuit(squin_circuit)
    sampler = stim_circuit.compile_sampler()
    samples = np.array(sampler.sample(shots=shots))

    # average number of 1s per shot (not a probability of any-error)
    err_frac = np.count_nonzero(samples) / len(samples)
    return err_frac, samples


In [ ]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit

rates = {
    "Global": dict(px=6.5e-05,  py=6.5e-05,  pz=6.5e-05),
    "Local":  dict(px=2.0e-04,  py=2.0e-04,  pz=2.0e-04),
    "Xtalk":  dict(px=2e-07,    py=2e-07,    pz=1.2e-06),
    "Gate":   dict(px=1.5e-02,  py=1.5e-02,  pz=1.5e-02),
    "Sitter": dict(px=3.066e-04,py=3.066e-04,pz=4.639e-04),
    "Mover":  dict(px=5.0e-04,  py=5.0e-04,  pz=5.0e-04),
}

def model_twozone_only(level, scale=1.0):
    """
    start with all zeros, enable only one channel.
    """
    # all zeros
    p = dict(
        global_px=0.0, global_py=0.0, global_pz=0.0,
        local_px=0.0, local_py=0.0, local_pz=0.0,
        local_unaddressed_px=0.0, local_unaddressed_py=0.0, local_unaddressed_pz=0.0,
        cz_paired_gate_px=0.0, cz_paired_gate_py=0.0, cz_paired_gate_pz=0.0,
        cz_unpaired_gate_px=0.0, cz_unpaired_gate_py=0.0, cz_unpaired_gate_pz=0.0,
        sitter_px=0.0, sitter_py=0.0, sitter_pz=0.0,
        mover_px=0.0, mover_py=0.0, mover_pz=0.0,
    )

    r = rates[level]
    if level == "Global":
        p["global_px"] = r["px"] * scale; p["global_py"] = r["py"] * scale; p["global_pz"] = r["pz"] * scale
    elif level == "Local":
        p["local_px"] = r["px"] * scale; p["local_py"] = r["py"] * scale; p["local_pz"] = r["pz"] * scale
    elif level == "Xtalk":
        p["local_unaddressed_px"] = r["px"] * scale
        p["local_unaddressed_py"] = r["py"] * scale
        p["local_unaddressed_pz"] = r["pz"] * scale
    elif level == "Gate":
        # treat CZ gate noise as the dominant high-rate channel
        p["cz_paired_gate_px"] = r["px"] * scale
        p["cz_paired_gate_py"] = r["py"] * scale
        p["cz_paired_gate_pz"] = r["pz"] * scale
        p["cz_unpaired_gate_px"] = r["px"] * scale
        p["cz_unpaired_gate_py"] = r["py"] * scale
        p["cz_unpaired_gate_pz"] = r["pz"] * scale
    elif level == "Sitter":
        p["sitter_px"] = r["px"] * scale; p["sitter_py"] = r["py"] * scale; p["sitter_pz"] = r["pz"] * scale
    elif level == "Mover":
        p["mover_px"] = r["px"] * scale; p["mover_py"] = r["py"] * scale; p["mover_pz"] = r["pz"] * scale

    return noise.GeminiTwoZoneNoiseModel(**p)


In [ ]:
from typing import Literal
from bloqade.types import Qubit

def steane_encode_zero(q: IList[Qubit, Literal[7]]):
    """Encodes 7 physical qubits into a single logical |0> state."""
    squin.h(q[0])
    squin.h(q[1])
    squin.h(q[3])
    squin.cx(q[0], q[4])
    squin.cx(q[1], q[2])
    squin.cx(q[3], q[5])
    squin.cx(q[0], q[6])
    squin.cx(q[3], q[4])
    squin.cx(q[1], q[5])
    squin.cx(q[0], q[2])
    squin.cx(q[5], q[6])


def steane_encode_plus(q: IList[Qubit, Literal[7]]):
    steane_encode_zero(q)
    for i in range(7):
        squin.h(q[i])



@squin.kernel
def a3_circuit():
    q = squin.qalloc(22)

    data = IList([q[0], q[1], q[2], q[3], q[4], q[5], q[6]])
    anc1 = IList([q[8], q[9], q[10], q[11], q[12], q[13], q[14]])   # |+>L
    anc2 = IList([q[15], q[16], q[17], q[18], q[19], q[20], q[21]]) # |0>L

    # Optional: start clean
    squin.broadcast.reset(data)
    squin.broadcast.reset(anc1)
    squin.broadcast.reset(anc2)

    steane_encode_zero(data)   # encode data as |0>_L baseline

    # ancilla 1 as |+>_L to read Z errors (X-syndrome)
    steane_encode_plus(anc1)

    # Transversal CNOT: data -> anc1
    for i in range(7):
        squin.cx(data[i], anc1[i])

    # ancilla 2 as |0>_L to read X errors (Z-syndrome)
    steane_encode_zero(anc2)

    # Transversal CNOT: anc2 -> data
    for i in range(7):
        squin.cx(anc2[i], data[i])

    # Hadamards then measure anc2 (X-basis)
    for i in range(7):
        squin.h(anc2[i])

    # Measure both ancilla blocks (14 bits total)
    for i in range(7):
        squin.measure(anc1[i])
    for i in range(7):
        squin.measure(anc2[i])

@squin.kernel
def noise_test_circuit():
    """Test circuit for Fault-Tolerant syndrome extraction."""
    q = squin.qalloc(22)
    
    
    steane_encode_plus(q[8:15])
    for i in range(7):
        squin.cx(q[i], q[i+8])

    steane_encode_zero(q[15:22])
    for i in range(7):
        squin.cx(q[i+15], q[i])

    for i in range(7):
        squin.h(q[i+15])
    
    for i in range(8, 22):
        squin.measure(q[i])

In [89]:
import numpy as np
import bloqade.stim
from bloqade.cirq_utils import load_circuit

def run_error_fraction(cirq_circuit, shots=500):
    squin_circuit = load_circuit(cirq_circuit)
    stim_circuit = bloqade.stim.Circuit(squin_circuit)
    sampler = stim_circuit.compile_sampler()
    samples = np.array(sampler.sample(shots=shots))

    # average number of 1s per shot
    err_metric = np.count_nonzero(samples) / len(samples)
    return err_metric, samples


In [90]:
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import noise
import warnings
warnings.filterwarnings("ignore", message=r"Assumed Only single qubit gates in the layer.*")

cirq_enc = emit_circuit(a3_circuit)
shots = 500

def onezone_model_only(kind):
    # all zeros
    kwargs = dict(
        global_px=0.0, global_py=0.0, global_pz=0.0,
        local_px=0.0, local_py=0.0, local_pz=0.0,
        local_unaddressed_px=0.0, local_unaddressed_py=0.0, local_unaddressed_pz=0.0,
        cz_paired_gate_px=0.0, cz_paired_gate_py=0.0, cz_paired_gate_pz=0.0,
        cz_unpaired_gate_px=0.0, cz_unpaired_gate_py=0.0, cz_unpaired_gate_pz=0.0,
    )

    if kind == "Global":
        kwargs.update(global_px=6.5e-05, global_py=6.5e-05, global_pz=6.5e-05)
    elif kind == "Local":
        kwargs.update(local_px=2.0e-04, local_py=2.0e-04, local_pz=2.0e-04)
    elif kind == "Xtalk":
        kwargs.update(local_unaddressed_px=2e-07, local_unaddressed_py=2e-07, local_unaddressed_pz=1.2e-06)
    elif kind == "Gate":
        kwargs.update(
            cz_paired_gate_px=1.5e-02, cz_paired_gate_py=1.5e-02, cz_paired_gate_pz=1.5e-02,
            cz_unpaired_gate_px=1.5e-02, cz_unpaired_gate_py=1.5e-02, cz_unpaired_gate_pz=1.5e-02,
        )
    else:
        raise ValueError("kind must be Global/Local/Xtalk/Gate")

    return noise.GeminiOneZoneNoiseModel(**kwargs)

kinds = ["Global", "Local", "Xtalk", "Gate"]
for k in kinds:
    print("Simulating", k)
    cirq_noisy = noise.transform_circuit(cirq_enc, model=onezone_model_only(k))
    metric, samples = run_error_fraction(cirq_noisy, shots=shots)
    print("  samples shape:", samples.shape, "metric:", metric)


Simulating Global
  samples shape: (500, 14) metric: 7.07
Simulating Local
  samples shape: (500, 14) metric: 7.102
Simulating Xtalk
  samples shape: (500, 14) metric: 7.162
Simulating Gate
  samples shape: (500, 14) metric: 6.946
